In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import *
from os import listdir
from os.path import isfile, join
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [2]:
# dbutils.fs.rm("dbfs:/databricks-datasets/sms_spam_collection/data-001/smsData.csv")
# dbutils.fs.put( "dbfs:/databricks-datasets/spamData.txt", "file:/Users/aptr2/OneDrive/Desktop/BigDataIquest/Assignments/SpamHam/SMSSpamCollection.txt")

# #print(dbutils.fs.head("dbfs:/databricks-datasets/sms_spam_collection/README.md"))
# display(dbutils.fs.ls("dbfs:/databricks-datasets/"))

In [3]:
display(dbutils.fs.ls("dbfs:/databricks-datasets/sms_spam_collection/data-001/"))

path,name,size
dbfs:/databricks-datasets/sms_spam_collection/data-001/smsData.csv,smsData.csv,113674


In [4]:
path = "dbfs:/databricks-datasets/sms_spam_collection/data-001/smsData.csv"
df = spark.read.option("delimiter","\t").csv(path).toDF("spam","message")

In [5]:
df.printSchema()

root
-- spam: string (nullable = true)
-- message: string (nullable = true)

In [6]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
tokenizer = Tokenizer(inputCol="message", outputCol="words")
tokenized = tokenizer.transform(df)

In [7]:
# from pyspark.ml.feature import StopWordsRemover
# remover = StopWordsRemover(inputCol="words", outputCol="filtered")
# stopwordsRemoved = remover.transform(tokenized).show(truncate=False)
# #adding "-" to the existing stopwords list  
# # newStopwordAdded = remover.getStopWords()
# # # print(type(newStopwordAdded))
# # newStopwordAdded.append("-")
# # remover2 = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords = newStopwordAdded)
# # stopwordsRemoved = remover2.transform(tokenized).show(truncate=False)
# # stopwordsRemovedNoNull = stopwordsRemoved.where(col("filtered").isNotNull())
# # stopwordsRemovedNoNull = stopwordsRemoved.filter(stopwordsRemoved.col("words").isNotNull())
# newrdd = stopwordsRemoved.na.drop()
# fit a CountVectorizerModel from the corpus.
# from pyspark.ml.feature import CountVectorizer
# cv = CountVectorizer(inputCol="filtered", outputCol="features")
# model = cv.fit(stopwordsRemoved)
# result = model.transform(stopwordsRemoved)
# result.show(truncate=False)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3813808036742777> in <module> () 
 10 # stopwordsRemovedNoNull = stopwordsRemoved.where(col("filtered").isNotNull()) 
 11 # stopwordsRemovedNoNull = stopwordsRemoved.filter(stopwordsRemoved.col("words").isNotNull()) 
 ---> 12 newrdd = stopwordsRemoved . na . drop ( ) 

 AttributeError : 'NoneType' object has no attribute 'na'

In [8]:
# fit a CountVectorizerModel from the corpus.
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="words", outputCol="features")
model = cv.fit(tokenized)
result = model.transform(tokenized)
result.show(truncate=False)

+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
spam|message |words |features |
+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
ham |Ok... But they said i've got wisdom teeth hidden inside n mayb need 2 remove. |[ok..., but, they, said, i've, got, wisdom, teeth, hidden, inside, n, mayb, need, 2, remove.] |(4800,[4,20,29,31,96,98,113,153,220,263,1025,2928,3090,3750,4697],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
ham |� thk of wat to eat tonight. |[�, thk, of, wat, to, eat, tonight.] |(4800,[0,10,26,64,67,170,581],[1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
ham |I dunno until when... Lets go learn pilates... |[i, dunno, until, when..., lets, go, learn, pilates...] |(4800,[2,17,112,206,1068,1376,4382,4797],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
spam|Someonone you know is trying to contact you via our dating service! To find out who it could be call from your mobile or landline 09064015307 BOX334SK38ch |[someonone, you, know, is, trying, to, contact, you, via, our, dating, service!, to, find, out, who, it, could, be, call, from, your, mobile, or, landline, 09064015307, box334sk38ch] |(4800,[0,5,6,7,15,19,39,45,51,60,71,75,88,108,118,143,247,288,426,899,967,1249,2081,2769,4433],[2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
ham |Ok c � then. |[ok, c, �, then.] |(4800,[10,34,69,1347],[1.0,1.0,1.0,1.0]) |
spam|URGENT! We are trying to contact U. Todays draw shows that you have won a �800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid12hrs only |[urgent!, we, are, trying, to, contact, u., todays, draw, shows, that, you, have, won, a, �800, prize, guaranteed., call, 09050003091, from, land, line., claim, c52., valid12hrs, only] |(4800,[0,3,5,6,22,25,35,39,46,59,72,77,88,94,116,127,161,247,259,283,285,328,394,542,1207,1455,1733],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
spam|Not heard from U4 a while. Call 4 rude chat private line 01223585334 to cum. Wan 2C pics of me gettin shagged then text PIX to 8552. 2End send STOP 8552 SAM xxx |[not, heard, from, u4, a, while., call, 4, rude, chat, private, line, 01223585334, to, cum., wan, 2c, pics, of, me, gettin, shagged, then, text, pix, to, 8552., 2end, send, stop, 8552, sam, xxx]|(4800,[0,3,5,9,11,24,26,38,39,58,74,91,139,198,214,312,498,902,1775,1997,2066,2167,2256,2742,2984,3108,3479,3509,3589,3662,4123,4363],[2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
ham |staff.science.nus.edu.sg/~phyhcmk/teaching/pc1323 |[staff.science.nus.edu.sg/~phyhcmk/teaching/pc1323] |(4800,[1901],[1.0]) |
ham |Thank god they are in bed! |[thank, god, they, are, in, bed!] |(4800,[16,35,98,1033,1518,2638],[1.0,1.0,1.0,1.0,1.0,1.0])

In [9]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="spam", outputCol="label")
indexedData = indexer.fit(result).transform(result)
indexedData.show()

+----+--------------------+--------------------+--------------------+-----+
spam| message| words| features|label|
+----+--------------------+--------------------+--------------------+-----+
 ham|Ok... But they sa...|[ok..., but, they...|(4800,[4,20,29,31...| 0.0|
 ham|� thk of wat to e...|[�, thk, of, wat,...|(4800,[0,10,26,64...| 0.0|
 ham|I dunno until whe...|[i, dunno, until,...|(4800,[2,17,112,2...| 0.0|
spam|Someonone you kno...|[someonone, you, ...|(4800,[0,5,6,7,15...| 1.0|
 ham| Ok c � then.| [ok, c, �, then.]|(4800,[10,34,69,1...| 0.0|
spam|URGENT! We are tr...|[urgent!, we, are...|(4800,[0,3,5,6,22...| 1.0|
spam|Not heard from U4...|[not, heard, from...|(4800,[0,3,5,9,11...| 1.0|
 ham|staff.science.nus...|[staff.science.nu...| (4800,[1901],[1.0])| 0.0|
 ham|Thank god they ar...|[thank, god, they...|(4800,[16,35,98,1...| 0.0|
 ham|Hey tmr meet at b...|[hey, tmr, meet, ...|(4800,[18,78,86,1...| 0.0|
spam|You are a winner ...|[you, are, a, win...|(4800,[0,3,5,6,15...| 1.0|
spam|URGENT! Your Mobi...|[urgent!, your, m...|(4800,[0,5,7,14,1...| 1.0|
spam|Loan for any purp...|[loan, for, any, ...|(4800,[5,6,15,21,...| 1.0|
 ham|Haha... Sounds cr...|[haha..., sounds,...|(4800,[27,112,202...| 0.0|
spam|You have won ?spa...|[you, have, won, ...|(4800,[0,3,6,15,2...| 1.0|
 ham|Sorry i din lock ...|[sorry, i, din, l...|(4800,[2,13,128,4...| 0.0|
 ham|Thanx but my birt...|[thanx, but, my, ...|(4800,[7,13,20,21...| 0.0|
spam|FREE for 1st week...|[free, for, 1st, ...|(4800,[0,9,12,21,...| 1.0|
spam|Congratulations -...|[congratulations,...|(4800,[0,1,3,4,5,...| 1.0|
 ham|Me n him so funny...|[me, n, him, so, ...|(4800,[11,23,31,1...| 0.0|
+----+--------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [10]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
train, test = indexedData.randomSplit([0.7, 0.3], seed=12345)

In [11]:
from pyspark.ml.classification import LogisticRegression


lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(train)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

predictions = lrModel.transform(test)\
    .select("features", "label", "prediction")\
    .show(100)

Coefficients: (4800,[],[])
Intercept: -1.166358277200498
+--------------------+-----+----------+
 features|label|prediction|
+--------------------+-----+----------+
(4800,[18,695,739...| 0.0| 0.0|
 (4800,[4519],[1.0])| 0.0| 0.0|
(4800,[1,4,53,57,...| 0.0| 0.0|
(4800,[1,4,17,26,...| 0.0| 0.0|
(4800,[2,18,27,36...| 0.0| 0.0|
(4800,[27,29,34,6...| 0.0| 0.0|
(4800,[2,4,13,53,...| 0.0| 0.0|
(4800,[1,4,12,25,...| 0.0| 0.0|
(4800,[1,2,13,23,...| 0.0| 0.0|
(4800,[9,10,24,38...| 0.0| 0.0|
(4800,[26,40,126,...| 0.0| 0.0|
 (4800,[2639],[1.0])| 0.0| 0.0|
(4800,[6,35,490,2...| 0.0| 0.0|
(4800,[1,5,11,27,...| 0.0| 0.0|
(4800,[6,75,94,24...| 0.0| 0.0|
(4800,[194,256,58...| 0.0| 0.0|
(4800,[2,13,20,12...| 0.0| 0.0|
(4800,[20,27,28,5...| 0.0| 0.0|
(4800,[20,22,29,4...| 0.0| 0.0|
(4800,[2,9,27,32,...| 0.0| 0.0|
(4800,[9,11,16,55...| 0.0| 0.0|
(4800,[1,2,21,111...| 0.0| 0.0|
(4800,[1,26,47,41...| 0.0| 0.0|
(4800,[1,2,5,11,2...| 0.0| 0.0|
(4800,[10,13,27,4...| 0.0| 0.0|
(4800,[0,5,10,11,...| 0.0| 0.0|
(4800,[0,36,79,12...| 0.0| 0.0|
(4800,[0,8,13,15,...| 0.0| 0.0|
(4800,[1,4,44,86,...| 0.0| 0.0|
(4800,[71,230,268...| 0.0| 0.0|
(4800,[1,9,22,40,...| 0.0| 0.0|
(4800,[14,36,67,6...| 0.0| 0.0|
(4800,[1,2,32,101...| 0.0| 0.0|
(4800,[1,2,7,12,1...| 0.0| 0.0|
(4800,[1,36,57,67...| 0.0| 0.0|
(4800,[2,3,9,10,1...| 0.0| 0.0|
(4800,[22,23,40,4...| 0.0| 0.0|
(4800,[84,112,171...| 0.0| 0.0|
(4800,[2,17,61,87...| 0.0| 0.0|
(4800,[0,2,9,10,1...| 0.0| 0.0|
(4800,[2089,2123]...| 0.0| 0.0|
(4800,[13,537,180...| 0.0| 0.0|
(4800,[1,2,11,12,...| 0.0| 0.0|
(4800,[1,91,172,5...| 0.0| 0.0|
(4800,[1,2,4,7,13...| 0.0| 0.0|
(4800,[1,8,41,87,...| 0.0| 0.0|
(4800,[2,23,96,98...| 0.0| 0.0|
(4800,[0,8,10,23,...| 0.0| 0.0|
(4800,[2,14,18,20...| 0.0| 0.0|
(4800,[16,17,29,3...| 0.0| 0.0|
(4800,[0,36,53,23...| 0.0| 0.0|
(4800,[4,7,13,20,...| 0.0| 0.0|
(4800,[10,17,23,2...| 0.0| 0.0|
(4800,[13,110,418...| 0.0| 0.0|
(4800,[13,566,259...| 0.0| 0.0|
(4800,[1,2,3,9,26...| 0.0| 0.0|
(4800,[18,32,65,7...| 0.0| 0.0|
(4800,[1,9,14,38,...| 0.0| 0.0|
(4800,[1,23,106,7...| 0.0| 0.0|
(4800,[1,4,11,57,...| 0.0| 0.0|
(4800,[179,274,30...| 0.0| 0.0|
(4800,[12,56,66,4...| 0.0| 0.0|
(4800,[6,25,125,4...| 0.0| 0.0|
(4800,[0,3,6,25,3...| 0.0| 0.0|
(4800,[2,32,59,12...| 0.0| 0.0|
(4800,[10,66,70,9...| 0.0| 0.0|
 (4800,[4224],[1.0])| 0.0| 0.0|
(4800,[2,4,12,16,...| 0.0| 0.0|
(4800,[2,12,13,29...| 0.0| 0.0|
(4800,[7,70,78,19...| 0.0| 0.0|
(4800,[0,1,26,36,...| 0.0| 0.0|
(4800,[9,18,56,78...| 0.0| 0.0|
(4800,[78,418,977...| 0.0| 0.0|
(4800,[0,2,6,8,17...| 0.0| 0.0|
(4800,[4,8,14,78,...| 0.0| 0.0|
(4800,[1,3,9,10,1...| 0.0| 0.0|
(4800,[0,14,17,22...| 0.0| 0.0|
(4800,[7,14,17,19...| 0.0| 0.0|
(4800,[1,2,3,11,3...| 0.0| 0.0|
(4800,[4,8,14,44,...| 0.0| 0.0|
(4800,[0,2,7,10,2...| 0.0| 0.0|
(4800,[7,76,196,7...| 0.0| 0.0|
(4800,[2,9,20,31,...| 0.0| 0.0|
(4800,[1,3,16,23,...| 0.0| 0.0|
(4800,[5,7,13,19,...| 0.0| 0.0|
(4800,[1,3,4,8,11...| 0.0| 0.0|
(4800,[1,2,7,18,2...| 0.0| 0.0|
(4800,[0,7,8,21,2...| 0.0| 0.0|
(4800,[1,38,92,11...| 0.0| 0.0|
(4800,[1,2,4,11,2...| 0.0| 0.0|
(4800,[0,10,47,80...| 0.0| 0.0|
(4800,[0,10,47,48...| 0.0| 0.0|
(4800,[1,2,5,7,16...| 0.0| 0.0|
(4800,[12,1731,43...| 0.0| 0.0|
(4800,[70,556,817...| 0.0| 0.0|
(4800,[0,2,9,18,2...| 0.0| 0.0|
(4800,[1,23,131,1...| 0.0| 0.0|
(4800,[23,295,363...| 0.0| 0.0|
(4800,[122,223,36...| 0.0| 0.0|
(4800,[1,2,4,12,1...| 0.0| 0.0|
+--------------------+-----+----------+
only showing top 100 rows

In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
# testRDD = test.rdd 
# predictionAndLabels = testRDD.map(lambda lp: (float(model.predict(lp.features)), lp.label))
# # Evaluate model
# metrics = BinaryClassificationMetrics(predictionAndLabels)
# f1Score = metrics.fMeasure()
# print(f1Score)
evaluator = BinaryClassificationEvaluator(labelCol = "label", rawPredictionCol="prediction")

evaluator.evaluate(predictions)